In [2]:
import sqlalchemy
print (sqlalchemy.__version__)

1.1.12


In [3]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///:memory:', echo=True)

In [5]:
print (engine.execute("select 1").scalar())

2017-08-03 04:07:08,389 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2017-08-03 04:07:08,390 INFO sqlalchemy.engine.base.Engine ()
2017-08-03 04:07:08,392 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2017-08-03 04:07:08,393 INFO sqlalchemy.engine.base.Engine ()
2017-08-03 04:07:08,394 INFO sqlalchemy.engine.base.Engine select 1
2017-08-03 04:07:08,395 INFO sqlalchemy.engine.base.Engine ()
1


In [6]:
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [7]:
from sqlalchemy import Column, Integer, String
class User(Base):
    __tablename__ = 'users'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    password = Column(String)
    def __init__(self, name, fullname, password):
        self.name = name
        self.fullname = fullname
        self.password = password
    def __repr__(self):
        return "<User('%s', '%s', '%s')>" % (self.name, self.fullname, self.password)

In [8]:
User.__table__


Table('users', MetaData(bind=None), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('name', String(), table=<users>), Column('fullname', String(), table=<users>), Column('password', String(), table=<users>), schema=None)

In [9]:
User.__mapper__

<Mapper at 0x1f333099ef0; User>

In [10]:
Base.metadata.create_all(engine)

2017-08-03 04:08:07,343 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2017-08-03 04:08:07,344 INFO sqlalchemy.engine.base.Engine ()
2017-08-03 04:08:07,346 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	password VARCHAR, 
	PRIMARY KEY (id)
)


2017-08-03 04:08:07,347 INFO sqlalchemy.engine.base.Engine ()
2017-08-03 04:08:07,348 INFO sqlalchemy.engine.base.Engine COMMIT


In [11]:
Column(String(50))

Column(None, String(length=50), table=None)

In [12]:
from sqlalchemy import Sequence
Column(Integer, Sequence('user_id_seq'), primary_key=True)

Column(None, Integer(), table=None, primary_key=True, nullable=False, default=Sequence('user_id_seq'))

In [13]:
class User(Base):
    __tablename__ = 'users'
    id = Column(Integer, Sequence('user_id_seq'), primary_key=True)
    name = Column(String(50))
    fullname = Column(String(50))
    password = Column(String(12))
    def __init__(self, name, fullname, password):
        self.name = name
        self.fullname = fullname
        self.password = password
    def __repr__(self):
        return "<User('%s', '%s', '%s')>" % (self.name, self.fullname, self.password)

C:\Users\Jihee\Miniconda3\envs\ml_python\lib\site-packages\sqlalchemy\ext\declarative\clsregistry.py:120: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.User, and will be replaced in the string-lookup table.
  item.__name__


InvalidRequestError: Table 'users' is already defined for this MetaData instance.  Specify 'extend_existing=True' to redefine options and columns on an existing Table object.

In [15]:
ed_user = User('haruair', 'Edward Kim', '1234')


In [17]:

ed_user.name        # 'haruair'


'haruair'

In [19]:
       # 'haruair'
ed_user.password    # '1234'


'1234'

In [20]:

str(ed_user.id)  

'None'

In [21]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)

In [22]:
Session = sessionmaker()

In [23]:
Session.configure(bind=engine)

In [24]:
session = Session()

In [25]:
ed_user= User('haruair', 'Edward Kim', '1234')
session.add(ed_user)

In [26]:
our_user = session.query(User).filter_by(name='haruair').first()
our_user 

2017-08-03 04:11:22,402 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-08-03 04:11:22,404 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2017-08-03 04:11:22,405 INFO sqlalchemy.engine.base.Engine ('haruair', 'Edward Kim', '1234')
2017-08-03 04:11:22,407 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?
2017-08-03 04:11:22,408 INFO sqlalchemy.engine.base.Engine ('haruair', 1, 0)


<User('haruair', 'Edward Kim', '1234')>

In [27]:
ed_user is our_user

True

In [28]:
session.add_all([
    User('wendy', 'Wendy Williams', 'foobar'),
    User('mary', 'Mary Contrary', 'xxg527'),
    User('fred', 'Fred Flinstone', 'blar')])

In [29]:
ed_user.password = 'test1234'

In [30]:
session.dirty 

IdentitySet([<User('haruair', 'Edward Kim', 'test1234')>])

In [31]:
session.new
# IdentitySet([<User('mary', 'Mary Contrary', 'xxg527')>,
#              <User('wendy', 'Wendy Williams', 'foobar')>,
#              <User('fred', 'Fred Flinstone', 'blar')>])

IdentitySet([<User('fred', 'Fred Flinstone', 'blar')>, <User('mary', 'Mary Contrary', 'xxg527')>, <User('wendy', 'Wendy Williams', 'foobar')>])

In [32]:
session.commit()

2017-08-03 05:05:56,109 INFO sqlalchemy.engine.base.Engine UPDATE users SET password=? WHERE users.id = ?
2017-08-03 05:05:56,111 INFO sqlalchemy.engine.base.Engine ('test1234', 1)
2017-08-03 05:05:56,112 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2017-08-03 05:05:56,113 INFO sqlalchemy.engine.base.Engine ('wendy', 'Wendy Williams', 'foobar')
2017-08-03 05:05:56,114 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2017-08-03 05:05:56,115 INFO sqlalchemy.engine.base.Engine ('mary', 'Mary Contrary', 'xxg527')
2017-08-03 05:05:56,116 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2017-08-03 05:05:56,117 INFO sqlalchemy.engine.base.Engine ('fred', 'Fred Flinstone', 'blar')
2017-08-03 05:05:56,118 INFO sqlalchemy.engine.base.Engine COMMIT


In [33]:
ed_user.id

2017-08-03 05:06:04,940 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-08-03 05:06:04,942 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.id = ?
2017-08-03 05:06:04,943 INFO sqlalchemy.engine.base.Engine (1,)


1

In [34]:
ed_user.name = 'edkim'

In [35]:
fake_user = User('fakeuser', 'Invalid', '12345')
session.add(fake_user)

In [36]:
session.query(User).filter(User.name.in_(['edkim', 'fakeuser'])).all()


2017-08-03 05:08:42,952 INFO sqlalchemy.engine.base.Engine UPDATE users SET name=? WHERE users.id = ?
2017-08-03 05:08:42,953 INFO sqlalchemy.engine.base.Engine ('edkim', 1)
2017-08-03 05:08:42,954 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2017-08-03 05:08:42,955 INFO sqlalchemy.engine.base.Engine ('fakeuser', 'Invalid', '12345')
2017-08-03 05:08:42,956 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name IN (?, ?)
2017-08-03 05:08:42,957 INFO sqlalchemy.engine.base.Engine ('edkim', 'fakeuser')


[<User('edkim', 'Edward Kim', 'test1234')>,
 <User('fakeuser', 'Invalid', '12345')>]

In [38]:
for instance in session.query(User).order_by(User.id):
    print (instance.name, instance.fullname)

2017-08-03 05:09:54,455 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users ORDER BY users.id
2017-08-03 05:09:54,456 INFO sqlalchemy.engine.base.Engine ()
edkim Edward Kim
wendy Wendy Williams
mary Mary Contrary
fred Fred Flinstone
fakeuser Invalid


In [40]:
for row in session.query(User, User.name).all():
    print (row.User, row.name)

2017-08-03 05:11:04,838 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users
2017-08-03 05:11:04,839 INFO sqlalchemy.engine.base.Engine ()
<User('edkim', 'Edward Kim', 'test1234')> edkim
<User('wendy', 'Wendy Williams', 'foobar')> wendy
<User('mary', 'Mary Contrary', 'xxg527')> mary
<User('fred', 'Fred Flinstone', 'blar')> fred
<User('fakeuser', 'Invalid', '12345')> fakeuser


In [42]:
for row in session.query(User.name.label('name_label')).all():
    print (row.name_label)

2017-08-03 05:11:20,161 INFO sqlalchemy.engine.base.Engine SELECT users.name AS name_label 
FROM users
2017-08-03 05:11:20,163 INFO sqlalchemy.engine.base.Engine ()
edkim
wendy
mary
fred
fakeuser


In [44]:
from sqlalchemy.orm import aliased
user_alias = aliased(User, name='user_alias')
for row in session.query(user_alias, user_alias.name).all():
    print (row.user_alias)

2017-08-03 05:11:40,210 INFO sqlalchemy.engine.base.Engine SELECT user_alias.id AS user_alias_id, user_alias.name AS user_alias_name, user_alias.fullname AS user_alias_fullname, user_alias.password AS user_alias_password 
FROM users AS user_alias
2017-08-03 05:11:40,211 INFO sqlalchemy.engine.base.Engine ()
<User('edkim', 'Edward Kim', 'test1234')>
<User('wendy', 'Wendy Williams', 'foobar')>
<User('mary', 'Mary Contrary', 'xxg527')>
<User('fred', 'Fred Flinstone', 'blar')>
<User('fakeuser', 'Invalid', '12345')>


In [45]:
for user in session.query(User).order_by(User.id)[1:3]:
    print (user)

2017-08-03 05:12:48,134 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users ORDER BY users.id
 LIMIT ? OFFSET ?
2017-08-03 05:12:48,135 INFO sqlalchemy.engine.base.Engine (2, 1)
<User('wendy', 'Wendy Williams', 'foobar')>
<User('mary', 'Mary Contrary', 'xxg527')>


In [46]:
for name in session.query(User.name).filter_by(fullname='Edward Kim'):
    print (name)

2017-08-03 05:13:05,566 INFO sqlalchemy.engine.base.Engine SELECT users.name AS users_name 
FROM users 
WHERE users.fullname = ?
2017-08-03 05:13:05,568 INFO sqlalchemy.engine.base.Engine ('Edward Kim',)
('edkim',)


In [47]:
for name in session.query(User.name).filter(User.fullname=='Edward Kim'):
    print (name)

2017-08-03 05:13:14,878 INFO sqlalchemy.engine.base.Engine SELECT users.name AS users_name 
FROM users 
WHERE users.fullname = ?
2017-08-03 05:13:14,880 INFO sqlalchemy.engine.base.Engine ('Edward Kim',)
('edkim',)


In [57]:
for name in session.query(User).\
            filter(User.name=='haruair').\
            filter(User.fullname=='Edward Kim'):
    print (user)

2017-08-03 05:19:32,541 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name = ? AND users.fullname = ?
2017-08-03 05:19:32,542 INFO sqlalchemy.engine.base.Engine ('haruair', 'Edward Kim')


In [58]:
query = session.query(User).filter(User.name.like('%air')). order_by(User.id)
query.all()

2017-08-03 05:19:32,726 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name LIKE ? ORDER BY users.id
2017-08-03 05:19:32,727 INFO sqlalchemy.engine.base.Engine ('%air',)


[]

In [59]:
query.first()

2017-08-03 05:19:33,258 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name LIKE ? ORDER BY users.id
 LIMIT ? OFFSET ?
2017-08-03 05:19:33,259 INFO sqlalchemy.engine.base.Engine ('%air', 1, 0)


In [61]:
from sqlalchemy.orm.exc import MultipleResultsFound
try:
    user = query.one()
except MultipleResultsFound, e:
    print e
from sqlalchemy.orm.exc import NoResultFound
try:
    user = query.filter(User.id == 99).one()
except NoResultFound, e:
    print (e)

SyntaxError: invalid syntax (<ipython-input-61-0188bcb70d78>, line 4)